# Предобрабатываю данные (избавляюсь от дубликатов), агрегирую до года, соединяю таблицы

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
cd ..

c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\data


c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import polars as pl

In [3]:
trade19 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2019_V202501.csv')
trade20 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2020_V202501.csv')
trade21 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2021_V202501.csv')
trade22 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2022_V202501.csv')
country_codes = pl.read_csv('./raw_data/BACI_HS17_V202501/country_codes_V202501.csv')
sql = pl.SQLContext(register_globals=True)

In [4]:
#Структура данных. t, i , j , k --- ключ.
sql.execute('''select * from trade19 limit 1;''').collect()

t,i,j,k,v,q
i64,i64,i64,i64,f64,f64
2019,4,20,420229,0.471,0.001


In [5]:
#Проверяю, что в данных нет дублей
print(sql.execute('''select i, j, k, count(*) as cnt from trade19 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade20 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade21 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade22 group by 1, 2, 3 order by 4 desc limit 1;''').collect())

shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 246 ┆ 458 ┆ 621149 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 170 ┆ 528 ┆ 620192 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 724 ┆ 251 ┆ 600192 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 764 ┆ 56  ┆ 551599 ┆ 1   │
└─────┴─────┴────────┴─────┘


In [4]:
# Меняю типы данных на более экономные, где это возможно, и объединяю таблицы для трёх лет
trade_tmp = sql.execute(''' 
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade20
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade21
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade22
            ;
            ''').collect()
sql.register('trade_tmp', trade_tmp)
sql.execute('''select count(*) from trade_tmp;''').collect()

len
u32
34145929


In [5]:
# Удаляю лишние таблицы, чтобы не засоряли оперативку
import gc
del trade20
del trade21
del trade22
gc.collect()

23

In [6]:
# Джоиню данные за 2019 год, чтобы было потом на них нормировать
trade_tmp2 = sql.execute('''
            select 
                tr.t
                , tr.i
                , tr.j
                , tr.category    
                , tr.v
                , tr.q             
                , t19.v as v19
                , t19.q as q19    
            from trade_tmp tr
            inner join trade19 t19
                on tr.i = t19.i
                and tr.j = t19.j
                and tr.k = t19.k
            ;''').collect()
sql.register('trade_tmp2', trade_tmp2)
del trade19
del trade_tmp
gc.collect()

0

In [ ]:
# Материализовал тяжёлый запрос, чтобы больше не мучить компьютер
# trade_tmp2.write_parquet('preprocessed_data/trade_tmp2.parquet.gzip')

In [8]:
# Хотя в country_codes есть страны, у которых два типа country_code, но один iso3, это соответствует кейсам преемственности. 
# На интервале 2019 -- 2022 этого не возникает (т.е. можно считать, что iso3 --- PK)
sql.execute('''select country_iso3, count(*) from country_codes group by 1, order by 2 desc limit 4;''').collect()

country_iso3,len
str,u32
"""BEL""",2
"""DEU""",2
"""SDN""",2
"""KWT""",1


In [9]:
# У некоторых стран два country_code, но один iso3 код. Чтобы убрать дубли, группирую и суммирую
trade = sql.execute('''
    select 
        tr.t
        , cc_i.country_iso3 as i
        , cc_j.country_iso3 as j
        , tr.category    
        , v
        , q             
        , v19
        , q19
    from trade_tmp2 tr
    inner join country_codes cc_i
        on tr.i = cc_i.country_code 
    inner join country_codes cc_j
        on tr.j = cc_j.country_code                      
;''').collect()
sql.register('trade', trade)
sql.execute('''select * from trade limit 2;''').collect()

t,i,j,category,v,q,v19,q19
i16,str,str,str,f32,f32,f64,f64
2020,"""AFG""","""AUS""","""080211""",2.456,0.71,6.944,1.198
2020,"""AFG""","""AUS""","""080420""",1.398,1.331,0.87,0.8


In [10]:
trade.write_parquet('preprocessed_data/trade_hs6.parquet.gzip')

In [ ]:
# Агрегирую данные о торговле с разной гранулярностью гармонизированной системы
sql.execute('''
            select 
                t, i, j, left(category, 4) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs4.parquet.gzip')

In [12]:
sql.execute('''select 
                t, i, j, left(category, 2) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs2.parquet.gzip')

In [13]:
sql.execute('''select 
                t, i, j
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3
            ;''').collect().write_parquet('preprocessed_data/trade_hs0.parquet.gzip')

In [14]:
del trade_tmp2
del trade
gc.collect()

21

## Агрегирую заболеваемость и смертность

In [16]:
covid_tmp = pl.read_csv('raw_data/Our World In Data/owid-covid-data.csv')
sql.register('covid_tmp', covid_tmp)
sql.execute('''select * from covid_tmp limit 2;''').collect()

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
str,str,str,str,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,i64,str,str,str,str
"""AFG""","""Asia""","""Afghanistan""","""2020-01-05""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null
"""AFG""","""Asia""","""Afghanistan""","""2020-01-06""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null


In [ ]:
# Надо подумать, нет ли тут эндогенных пропусков (т.е. если у страны нулевая заболеваемость,
#  значит ли это, что она ничего не репортит, а не репортит 0)
sql.execute('''create table covid as
    select 
        iso_code as country
        , left(date, 4)::smallint as t
        , avg(new_cases_per_million) as cases
        , avg(new_deaths_per_million) as deaths
        , avg(new_tests_per_thousand) as tests
        /*Лучше воспрользоваться данными ООН, чтобы точно использовать доковидные данные
        , avg(population_density) as population_density
        , avg(median_age) as median_age 
        , avg(aged_65_older) as aged_65_older
        , avg(aged_70_older) as aged_70_older
        , avg(cardiovasc_death_rate) as cardiovasc_death_rate
        , avg(handwashing_facilities) as handwashing_facilities
        , avg(life_expectancy) as life_expectancy */
    from covid_tmp
    where date <= '2022-12-31'
    group by iso_code, left(date, 4)        
;''')
sql.execute('''select * from covid limit 2;''').collect()

country,t,cases,deaths,tests
str,i16,f64,f64,str
"""AIA""",2022,435.593671,1.350329,null
"""LCA""",2020,4.712348,0.077265,null


## Агрегирую коронавирусные ограничения

In [18]:
restrictions_tmp = pl.read_csv('raw_data/OxCGRT/OxCGRT_compact_national_v1.csv')
sql.register('restrictions_tmp', restrictions_tmp)
sql.execute('''select * from restrictions_tmp 
            -- where `E3_Fiscal measures` is not null and `E3_Fiscal measures` > 0 
            limit 2;''').collect()

CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,C3M_Cancel public events,C3M_Flag,C4M_Restrictions on gatherings,C4M_Flag,C5M_Close public transport,C5M_Flag,C6M_Stay at home requirements,C6M_Flag,C7M_Restrictions on internal movement,C7M_Flag,C8EV_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6M_Facial Coverings,H6M_Flag,H7_Vaccination policy,H7_Flag,H8M_Protection of elderly people,H8M_Flag,V1_Vaccine Prioritisation (summary),V2A_Vaccine Availability (summary),V2B_Vaccine age eligibility/availability age floor (general population summary),V2C_Vaccine age eligibility/availability age floor (at risk summary),V2D_Medically/ clinically vulnerable (Non-elderly),V2E_Education,V2F_Frontline workers (non healthcare),V2G_Frontline workers (healthcare),V3_Vaccine Financial Support (summary),V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
str,str,str,str,str,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,str,f64,i64,f64,i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,str,f64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,str,i64,i64,str,f64,f64,f64,f64,f64
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200101,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200102,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0


In [ ]:
sql.execute('''create table restrictions as
    select 
        CountryCode as country
        , left(Date::varchar, 4)::smallint as t
        , avg(`C8EV_International travel controls`)::float4 as C8EV_International_travel_controls
        , avg(`E1_Income support`)::float4 as E1_Income_support
        , avg(`E2_Debt/contract relief`)::float4 as E2_Debt_contract_relief
        , avg(`E3_Fiscal measures`)::float4 as E3_Fiscal_measures
        , avg(`E4_International support`)::float4 as E4_International_support
        , avg(`H6M_Facial Coverings`)::float4 as H6M_Facial_Coverings
        , avg(`StringencyIndex_Average`)::float4 as StringencyIndex_Average
        , avg(`GovernmentResponseIndex_Average`)::float4 as GovernmentResponseIndex_Average
        , avg(`ContainmentHealthIndex_Average`)::float4 as ContainmentHealthIndex_Average
        , avg(`EconomicSupportIndex`)::float4 as EconomicSupportIndex
    from restrictions_tmp
    group by CountryCode, left(Date::varchar, 4)::smallint           
;''')
sql.execute('''select * from restrictions limit 10;''').collect()

country,t,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H6M_Facial_Coverings,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
str,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""SOM""",2021,1.328767,0.0,0.805479,0.0,0.0,2.838356,35.55463,40.044685,42.891125,20.136986
"""IRQ""",2020,2.407104,0.625683,0.625683,1.374136e6,0.0,2.797814,64.680519,53.440929,56.606529,31.284153
"""AFG""",2020,1.461749,0.0,0.084699,0.0,107650.273438,1.743169,41.487267,33.816692,38.345192,2.117486
"""KOR""",2021,2.435616,1.0,0.0,5.7709252e7,0.0,3.0,51.278656,56.760632,61.298466,25.0
"""TLS""",2020,3.10929,1.065574,0.579235,236625.109375,0.0,1.562842,31.876421,35.320518,34.490601,41.12022
"""BMU""",2022,0.868493,0.805479,1.610959,null,null,2.243836,23.291725,42.869781,40.366932,60.410957
"""PSE""",2020,3.295082,0.0,1.016393,0.0,0.0,0.046448,66.277298,47.251995,50.372894,25.409836
"""DOM""",2022,0.306849,0.0,0.0,null,null,0.852055,11.384027,23.000219,26.289343,0.0
"""KEN""",2020,2.322404,0.540984,0.644809,4.66864e6,0.0,2.084699,58.033062,48.092949,50.72858,29.644808


## Агрегирую инструментальные переменные

In [20]:
temperature_tmp = pl.read_csv('raw_data/Our World In Data/average-monthly-surface-temperature.csv')
sql.register('temperature_tmp', temperature_tmp)
sql.execute('''select * from temperature_tmp limit 2;''').collect()

Entity,Code,year,Day,Average surface temperature,Average surface temperature_duplicated_0
str,str,i64,str,f64,f64
"""Afghanistan""","""AFG""",1940,"""1940-01-15""",-2.032494,11.327695
"""Afghanistan""","""AFG""",1940,"""1940-02-15""",-0.733503,11.327695


In [ ]:
sql.execute('''create table temperature as
    select 
        Code as country
        , avg(`Average surface temperature`) as average_surface_temperature
    from temperature_tmp
    where year >= 2000 and year <= 2019
    group by Code               
;''')
sql.execute('''select * from temperature limit 2;''').collect()

country,average_surface_temperature
str,f64
"""COK""",24.407138
"""HMD""",1.65656


In [22]:
median_age_tmp = pl.read_excel('raw_data/United Nations/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx')
sql.register('median_age_tmp', median_age_tmp)
sql.execute('''select * from median_age_tmp where `ISO3 Alpha-code` <> '' limit 2;''').collect()

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 6, falling back to string


Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",Population Change (thousands),Population Growth Rate (percentage),Population Annual Doubling Time (years),Births (thousands),Births by women aged 15 to 19 (thousands),"Crude Birth Rate (births per 1,000 population)",Total Fertility Rate (live births per woman),Net Reproduction Rate (surviving daughters per woman),Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),Total Deaths (thousands),Male Deaths (thousands),Female Deaths (thousands),"Crude Death Rate (deaths per 1,000 population)","Life Expectancy at Birth, both sexes (years)",Male Life Expectancy at Birth (years),Female Life Expectancy at Birth (years),"Life Expectancy at Age 15, both sexes (years)",Male Life Expectancy at Age 15 (years),Female Life Expectancy at Age 15 (years),"Life Expectancy at Age 65, both sexes (years)",Male Life Expectancy at Age 65 (years),Female Life Expectancy at Age 65 (years),"Life Expectancy at Age 80, both sexes (years)",Male Life Expectancy at Age 80 (years),Female Life Expectancy at Age 80 (years),"Infant Deaths, under age 1 (thousands)","Infant Mortality Rate (infant deaths per 1,000 live births)",Live Births Surviving to Age 1 (thousands),"Under-Five Deaths, under age 5 (thousands)","Under-Five Mortality (deaths under age 5 per 1,000 live births)","Mortality before Age 40, both sexes (deaths under age 40 per 1,000 live births)","Male Mortality before Age 40 (deaths under age 40 per 1,000 male live births)","Female Mortality before Age 40 (deaths under age 40 per 1,000 female live births)","Mortality before Age 60, both sexes (deaths under age 60 per 1,000 live births)","Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
i64,str,str,str,i64,str,str,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2595,"""Estimates""","""Burundi""","""""",108,"""BDI""","""BI""",108,"""Country/Area""",910,1950,"""2229.322""","""2254.938""","""1080.184""","""1174.755""","""86.895""","""91.95""","""18.305""","""64.551""","""28.627""","""51.232""","""2.272""","""30.508""","""117.211""","""11.582""","""51.98""","""6.923""","""2.072""","""30.674""","""102.5""","""52.66""","""26.76""","""25.899""","""23.353""","""40.938""","""39.571""","""42.245""","""44.916""","""43.636""","""46.095""","""11.048""","""10.594""","""11.423""","""4.728""","""4.454""","""4.937""","""17.777""","""162.866""","""104.421""","""28.358""","""276.13""","""457.62""","""471.554""","""444.227""","""614.352""","""639.661""","""590.518""","""290.807""","""309.954""","""273.27""","

In [ ]:
sql.execute('''create table median_age_table as
    select country, median_age
    from (         
    select distinct on(country)
        `ISO3 Alpha-code` as country
        , Year as year
        , `Median Age, as of 1 July (years)`::float4 as median_age
    from median_age_tmp
    where `ISO3 Alpha-code` <> '' 
            and Year <= 2019       
    order by country, year desc 
            ) sub       
;''')
sql.execute('''select * from median_age_table order by 2 desc limit 10;''').collect()

country,median_age
str,f32
"""VAT""",60.292
"""MCO""",54.641998
"""SHN""",48.547001
"""JPN""",47.262001
"""MTQ""",47.029999
"""ITA""",45.998001
"""SMR""",45.332001
"""DEU""",44.903999
"""IMN""",44.831001


In [62]:
SARS_table = pl.read_excel('raw_data/WHO/SARS.xlsx')
sql.register('SARS_table', sql.execute('''select iso3 as country, Total as SARS_total_cases from SARS_table;''').collect())
sql.execute('''select * from SARS_table limit 2;''').collect()

country,SARS_total_cases
str,i64
"""AUT""",6
"""CAN""",251


In [25]:
density_and_mortality_tmp = pl.read_csv('raw_data/World Bank/pop_dens_and_mortality.csv')
sql.register('density_and_mortality_tmp', density_and_mortality_tmp)
sql.execute('''select * from density_and_mortality_tmp limit 2;''').collect()

Time,Time Code,Country Name,Country Code,Population density (people per sq. km of land area) [EN.POP.DNST],"Population, female (% of total population) [SP.POP.TOTL.FE.ZS]","Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]","Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]","Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]","Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]","Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]","Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]","Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]","Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]",Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS],Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]
i64,str,str,str,f64,f64,i64,str,str,str,str,f64,f64,f64,f64,f64
1990,"""YR1990""","""Afghanistan""","""AFG""",18.468424,50.096823,null,null,null,null,null,382.051,510.052,122.5,60.735996,12.86206
1990,"""YR1990""","""Albania""","""ALB""",119.946788,49.765495,null,null,null,null,null,66.103,127.21,35.2,20.653647,null


In [ ]:
sql.execute('''create table density_and_mortality as
    select country, population_density, female_ratio, maternal_mortality
            , cvd_mortality, pollution_mortality, poisoning_mortality
            , hygiene_mortality, female_mortality, male_mortality
            , infant_mortality, largest_city_share, urbanization
    from (        
    select distinct on(country)
        `Country Code` as country
        , Time as year
        , `Population density (people per sq. km of land area) [EN.POP.DNST]` population_density
        , `Population, female (% of total population) [SP.POP.TOTL.FE.ZS]` female_ratio
        , `Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]`::float4 maternal_mortality
        , `Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]`::float4 cvd_mortality
        , `Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]`::float4 pollution_mortality
        , `Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]`::float4 poisoning_mortality 
        , `Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]`::float4 hygiene_mortality
        , `Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]`::float4 female_mortality
        , `Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]`::float4 male_mortality
        , `Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]`::float4 infant_mortality
        , `Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS]`::float4 largest_city_share
        , `Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]`::float4 urbanization 
    from density_and_mortality_tmp
    where Time <= 2019        
    order by country, year desc
    ) sub        
;''')
sql.execute('''select * from density_and_mortality order by female_ratio limit 10;''').collect()

country,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization
str,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""QAT""",229.648129,26.710807,null,10.7,90.800003,0.3,0.4,37.437,46.422001,5.3,27.357042,null
"""ARE""",133.00176,35.48185,null,18.5,70.0,0.4,0.8,44.261002,68.244003,5.0,34.558586,62.610657
"""OMN""",14.834381,36.346578,null,21.5,104.0,0.9,1.6,53.222,76.462997,9.3,38.278755,32.706516
"""BHR""",1902.251282,37.517391,null,16.1,68.099998,0.3,0.8,48.797001,58.199001,6.0,45.268242,null
"""KWT""",249.288215,38.318484,null,11.9,45.200001,0.4,0.8,31.812,59.479,7.5,68.713997,68.713997
"""SAU""",13.985179,38.831635,null,20.9,91.099998,0.8,1.9,81.362999,100.896004,6.2,27.977003,54.159729
"""MDV""",1625.77,38.838087,null,11.6,32.0,0.0,2.3,41.924999,56.172001,6.0,null,null
"""SYC""",212.228261,45.395279,null,21.1,43.099998,0.5,8.4,79.568001,188.007996,12.8,null,null
"""PLW""",38.691304,45.982695,null,null,null,null,null,null,null,22.9,null,null


## Переменные, которые не входят в первый приоритет

In [27]:
household_size_tmp = pl.read_excel('raw_data/United Nations/undesa_pd_2022_hh-size-composition.xlsx')
sql.register('household_size_tmp', household_size_tmp)
sql.execute('''select * from household_size_tmp limit 2;''').collect()

Could not determine dtype for column 9, falling back to string
Could not determine dtype for column 10, falling back to string


Country or area,ISO Code,Data source category,Reference date (dd/mm/yyyy),Average household size (number of members),1 member,2-3 members,4-5 members,6 or more members
str,i64,str,str,f64,f64,f64,f64,f64
"""Afghanistan""",4,"""MICS""","""2010-01-07 00:00:00""",7.75,0.15,7.41,18.59,73.85
"""Afghanistan""",4,"""DHS""","""19/10/2015""",8.04,0.19,6.45,16.84,76.52


In [48]:
sql.execute('''create table household_size as
            select country, avg_hh_size, share_1_members, share_2_3_members, share_4_5_members
            from
            (select distinct on(country)
                cc.country_iso3 as country
                , case when `Reference date (dd/mm/yyyy)` like '__/__/____' then right(`Reference date (dd/mm/yyyy)`, 4)
                    else left(`Reference date (dd/mm/yyyy)`, 4) end ::int as year 
                , `Data source category` as data_source
                , `Average household size (number of members)`::float4 avg_hh_size
                , `1 member`::float4 as share_1_members
                , `2-3 members`::float4 as share_2_3_members
                , `4-5 members`::float4 as share_4_5_members
            from household_size_tmp hh
            inner join country_codes cc
                on hh.`ISO Code` = cc.country_code
            order by 1, 2 desc
            , data_source -- для детерминированности
            ) sub
;
''')
sql.execute('''select * from household_size order by 3 desc limit 10;''').collect()

country,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members
str,f32,f32,f32,f32
"""AUS""",2.55,null,null,null
"""JAM""",3.06,null,null,null
"""KNA""",null,null,null,null
"""NZL""",2.6,null,null,null
"""LVA""",2.26,41.080002,40.799999,14.41
"""FIN""",2.07,41.009998,44.459999,12.97
"""BLR""",2.16,37.959999,47.02,13.54
"""DEU""",2.14,37.27,48.09,13.21
"""AUT""",2.27,36.290001,44.93,16.49


## Соединяю информацию о коронавирусе, ограничениях и контрольных переменных в одну таблицу и экспортирую её

In [ ]:
sql.register('regressors_tmp',
            sql.execute('''
    select 
        *
    from covid c
    full outer join restrictions r
        using(country, t)
    ;''').collect().with_columns(pl.coalesce(['country', 'country:r']).alias("country")
                                    , pl.coalesce(['t', 't:r']).alias("t")).drop(['country:r', 't:r']))
regressors = sql.execute('''
    select * 
    from regressors_tmp r
    left join temperature t
        using(country)
    left join median_age_table
        using(country) 
    left join SARS_table
        using(country)
    left join density_and_mortality
        using(country) 
    left join household_size
        using(country)
    ;''').collect()
regressors

country,t,cases,deaths,tests,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H6M_Facial_Coverings,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,country:t,average_surface_temperature,country:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members
str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f64,str,f32,str,i64,str,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32
"""BMU""",2021,232.765781,4.102904,null,1.339726,1.641096,0.361644,700934.5625,0.0,2.123288,49.389946,56.705837,59.150932,39.589043,null,null,"""BMU""",43.743999,null,null,"""BMU""",1187.166667,50.701171,null,null,null,null,null,42.748001,98.474998,null,null,null,"""BMU""",2.26,34.09,48.389999,15.81
"""FLK""",2020,22.791077,0.0,null,null,null,null,null,null,null,null,null,null,null,"""FLK""",6.645397,"""FLK""",39.458,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MAR""",2020,31.862597,0.533094,null,2.896175,0.775956,1.240437,4.6284076e7,0.0,2.713115,61.154781,55.317104,56.017677,50.409836,"""MAR""",18.634238,"""MAR""",27.874001,null,null,"""MAR""",81.135779,49.505473,null,24.1,66.599998,0.7,4.6,85.392998,127.667999,16.700001,16.291014,21.930796,"""MAR""",4.58,7.2,26.18,37.939999
"""CRI""",2022,319.252767,0.928932,null,0.238356,0.0,0.468493,null,null,1.912329,19.169371,30.04896,32.667973,11.712329,"""CRI""",23.097835,"""CRI""",32.014,null,null,"""CRI""",97.915257,50.509616,null,9.5,17.799999,0.1,2.9,51.240002,103.413002,7.2,34.434021,27.573385,"""CRI""",3.21,12.07,48.799999,31.610001
"""SSD""",2021,2.810219,0.01789,null,1.652055,0.326027,0.0,0.0,0.0,2.364383,45.307178,41.671341,46.461479,8.150685,"""SSD""",27.990809,"""SSD""",16.594,null,null,"""SSD""",16.494523,50.809699,null,16.799999,134.5,2.3,68.099998,303.281006,360.368011,63.799999,18.598713,null,"""SSD""",5.98,3.15,14.56,29.370001
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KOR""",2021,29.354219,0.237644,null,2.435616,1.0,0.0,5.7709252e7,0.0,3.0,51.278656,56.760632,61.298466,25.0,"""KOR""",12.504002,"""KOR""",42.183998,"""KOR""",3,"""KOR""",530.377275,50.002248,null,7.3,18.9,0.2,5.2,22.07,57.021,2.7,23.634396,50.157372,"""KOR""",2.34,31.75,48.099998,19.27
"""RKS""",2020,null,null,null,1.483607,0.756831,1.540984,4.0164e6,0.0,1.967213,57.463444,49.54866,49.772949,47.984974,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MAC""",2020,null,null,null,2.73224,0.882514,1.765027,1.7112666e7,0.0,1.819672,29.509289,43.191994,39.906174,66.188522,null,null,"""MAC""",36.713001,"""MAC""",1,"""MAC""",20425.531915,53.410564,null,null,null,null,null,25.018999,49.986,null,95.54911,null,"""MAC""",3.07,15.13,48.16,31.059999


In [68]:
regressors.write_parquet('preprocessed_data/regressors.parquet.gzip')